# Falência em Taiwan 🇹🇼

In [ ]:
# Import libraries here
import gzip
import json
import pickle

import ipywidgets as widgets
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from ipywidgets import interact
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    classification_report,
    confusion_matrix,
)
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from teaching_tools.widgets import ConfusionMatrixWidget

# Preparando Dados

## Importação

### Exercício:
Carregue o conteúdo do arquivo `"data/taiwan-bankruptcy-data.json.gz"` e atribua-o à variável `taiwan_data`.

In [ ]:
# Load data file


### Exercício:
Extraia os nomes das chaves de <code>taiwan_data</code> e atribua-os à variável <code>taiwan_data_keys</code>.

__Dica:__ Os dados nesta tarefa podem estar organizados de forma diferente dos dados do projeto, portanto, certifique-se de inspecioná-los primeiro.

### Exercício:
Calcule quantas empresas estão em `taiwan_data` e atribua o resultado à variável `n_companies`.

In [ ]:
n_companies = len(taiwan_data["observations"])
print(n_companies)

### Exercício:
Calcule o número de features associadas a cada empresa e atribua o resultado à variável `n_features`.

In [ ]:
n_features = ...
print(n_features)

### Exercício:
Crie uma função `wrangle` que receba como entrada o caminho de um arquivo JSON compactado e retorne o conteúdo do arquivo como um DataFrame. Certifique-se de que o índice do DataFrame contenha o ID das empresas. Quando sua função estiver completa, use-a para carregar os dados no DataFrame `df`.

In [ ]:
def wrangle():
  pass

In [ ]:
df = ...
print("df shape:", df.shape)
df.head()

## Explorar

### Exercício:
Existe algum dado ausente no conjunto de dados? Crie uma Series onde o índice contenha o nome das colunas em `df` e os valores sejam o número de valores <code>NaN</code> em cada coluna. Atribua o resultado à variável <code>nans_by_col</code>. Nem a própria Series nem seu índice precisam de um nome.

In [ ]:
nans_by_col = ...
print("nans_by_col shape:", nans_by_col.shape)
nans_by_col.head()

### Exercício:
Os dados estão desbalanceados? Crie um gráfico de barras que mostre a contagem de valores normalizados para a coluna `df["bankrupt"]`. Certifique-se de rotular o eixo x como `"Bankrupt"`, o eixo y como `"Frequency"`, e usar o título `"Class Balance"`.

## Divisão:

### Exercício:
Crie sua matriz de features `X` e o vetor alvo `y`. Seu alvo é `"bankrupt"`.

In [ ]:
target = ...
X = ...
y = ...
print("X shape:", X.shape)
print("y shape:", y.shape)

### Exercício:
Divida seu conjunto de dados em conjuntos de treinamento e teste usando uma divisão aleatória. Seu conjunto de teste deve ser 20% dos seus dados. Certifique-se de definir `random_state` como `42`.

In [ ]:
X_train, X_test, y_train, y_test = ...
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

## Resample

### Exercício:
Crie uma nova matriz de features `X_train_over` e um vetor alvo `y_train_over` realizando a sobre-amostragem aleatória nos dados de treinamento. Certifique-se de definir `random_state` como `42`.

In [ ]:
over_sampler = ...
X_train_over, y_train_over = ...
print("X_train_over shape:", X_train_over.shape)
X_train_over.head()

# Construindo Model

## Iterar

### Exercício:
Crie um classificador <code>clf</code> que possa ser treinado em `(X_train_over, y_train_over)`. Você pode usar qualquer um dos novos preditores em conjunto que aprendeu neste projeto.

In [ ]:
clf = ...

### Exercício:
Realize a validação cruzada com seu classificador usando os dados de treinamento sobre-amostrados e atribua seus resultados à variável <code>cv_scores</code>. Certifique-se de definir o argumento <code>cv</code> como 5.

__Dica:__ Use suas pontuações de validação cruzada para avaliar diferentes classificadores. Escolha o que lhe der os melhores resultados.

In [ ]:
cv_scores = ...
print(cv_scores)

### Exercício:
Crie um dicionário <code>params</code> com a faixa de hiperparâmetros que você deseja avaliar para seu classificador. Se você não tiver certeza de quais hiperparâmetros ajustar, consulte a documentação do scikit-learn para seu preditor em busca de ideias.

__Dica:__ Se o classificador que você construiu for apenas um preditor (não um pipeline com várias etapas), você não precisa incluir o nome da etapa nas chaves do seu dicionário <code>params</code>.

Por exemplo, se seu classificador for apenas uma floresta aleatória (não um pipeline contendo uma floresta aleatória), você acessaria o número de estimadores usando <code>"n_estimators"</code>, e não <code>"randomforestclassifier__n_estimators"</code>.

In [ ]:
params = ...

### Exercício:
Crie um <code>GridSearchCV</code> chamado `model` que inclua seu classificador e a grade de hiperparâmetros. Certifique-se de definir `cv` como 5, `n_jobs` como -1 e `verbose` como 1.

In [ ]:
model = ...

### Exercício  
Ajuste seu modelo aos dados de treinamento **sobre-amostrados.**

In [ ]:
model.fit()

### Exercício:
Extraia os resultados da validação cruzada do seu modelo e carregue-os em um DataFrame chamado <code>cv_results</code>. Observando os resultados, qual conjunto de hiperparâmetros levou ao melhor desempenho?

In [ ]:
cv_results = ...
cv_results.head(5)

### Exercício:
Extraia os melhores hiperparâmetros do seu modelo e atribua-os à variável <code>best_params</code>.

In [ ]:
best_params = ...
print(best_params)

## Avaliar

### Exercício:
Teste a qualidade do seu modelo calculando as pontuações de precisão para os dados de treinamento e teste.

In [ ]:
acc_train = ...
acc_test = ...

print("Model Training Accuracy:", round(acc_train, 4))
print("Model Test Accuracy:", round(acc_test, 4))

### Exercício:
Plote uma matriz de confusão que mostre como seu modelo se desempenhou em seu conjunto de teste.

### Exercício:
Gere um relatório de classificação para o desempenho do seu modelo nos dados de teste e atribua-o à variável `class_report`.

In [ ]:
class_report = classification_report(y_test, model.predict(X_test))
print(class_report)

# Comunicar Resultados

### Exercício:
Crie um gráfico de barras horizontal com as 10 características mais importantes para o seu modelo. Certifique-se de rotular o eixo x como `"Gini Importance"`, o eixo y como `"Feature"` e usar o título `"Feature Importance"`.

### Exercício:
Salve seu modelo de melhor desempenho em um arquivo nomeado <code>"model-5-5.pkl"</code>.

In [ ]:
# Save model


### Exercício:
Abra o arquivo <code>my_predictor_assignment.py</code>. Adicione sua função `wrangle` e, em seguida, crie uma função `make_predictions` que recebe dois argumentos: `data_filepath` e <code>model_filepath</code>. Use a célula abaixo para testar seu módulo. Quando estiver satisfeito com o resultado, envie-o para o avaliador.

In [ ]:
# Import your module


# Generate predictions
y_test_pred = make_predictions(
    data_filepath="data/taiwan-bankruptcy-data-test-features.json.gz",
    model_filepath="model-5-5.pkl",
)

print("predictions shape:", y_test_pred.shape)
y_test_pred.head()

<div class="alert alert-info" role="alert">
    <b>Dica:</b> Se você receber um <code style="color:#E45E5C;background-color:#FEDDDE">ImportError</code> ao tentar importar <code>make_predictions</code> de <code>my_predictor_assignment</code>, tente reiniciar seu kernel. Vá ao menu <b>Kernel</b> e clique em <b>Reiniciar Kernel e Limpar Todas as Saídas</b>. Em seguida, execute apenas a célula acima. ☝️
</div>